In [1]:
from dotenv import dotenv_values
import requests
import json
import re
import pandas as pd

In [2]:
#import cosmos db credentials 
config = dotenv_values('credential.env')
ai_search_location = config['ai_search_location']
ai_search_key = config['ai_search_key']
ai_search_url = config['ai_search_url']
ai_search_index = 'oewg-speech-meeeting-index'
ai_search_name = 'oewg-meeting'
embedding_length = 768

In [3]:
#read csv file
df = pd.read_csv('/Users/pelumioluwaabiola/Desktop/Transcriptions/detailed_data/Meeting_5_Session_3.csv') 

In [4]:
#check if there are dulplicate values in id column and print the duplicate rows
duplicate = df[df.duplicated(['id'])]
duplicate

,id,Session,Meeting,Speaker,Text,TextEmbeddings,SpeakerEmbeddings


In [5]:
#read row by row and append to a list
data = []
for index, row in df.iterrows():
    row_dict = {}
    row_dict['id'] = row['id']
    row_dict['Session'] = str(row['Session'])
    row_dict['Meeting'] = str(row['Meeting'])
    row_dict['Speaker'] = row['Speaker']
    row_dict['Text'] = row['Text']
    #convert text embeddings to list
    row_dict['TextEmbeddings'] = json.loads(row['TextEmbeddings'])
    row_dict['SpeakerEmbeddings'] = json.loads(row['SpeakerEmbeddings'])
    row_dict['@search.action'] = 'upload'
    data.append(row_dict)

In [6]:
def insert_into_index(documents):
    """Uploads a list of 'documents' to Azure AI Search index."""

    url = f"https://{ai_search_name}.search.windows.net/indexes/{ai_search_index}/docs/index?api-version=2023-11-01"

    payload = json.dumps({"value": documents})
    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code == 200 or response.status_code == 201:
        return "Success"
    else:
        return f"Failure: {response.text}"

def make_safe_id(row_id: str):
    """Strips disallowed characters from row id for use as Azure AI search document ID."""
    return re.sub("[^0-9a-zA-Z_-]", "_", row_id)

#upload data in batchs of 1000
BATCH_SIZE = 1000
for i in range(0, len(data), BATCH_SIZE):
    batch = data[i:i+BATCH_SIZE]
    status = insert_into_index(batch)
    print(f'upload status: {status}')

upload status: Success
